# 2020 US Election Progress Analysis

https://www.theguardian.com/us-news/ng-interactive/2020/nov/07/us-election-2020-live-results-donald-trump-joe-biden-presidential-votes-pennsylvania-georgia-arizona-nevada

In [1]:
import requests
import pandas as pd
import datetime
import dateutil.parser
import json
import re
import glob
import sys
import locale
locale.setlocale(locale.LC_ALL, '')  # Use '' for auto, or force e.g. to 'en_US.UTF-8'

# gimme 100% width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

def format_percent(x):
    return f'{x*100:.2f}'

def flattened(l):
    return [item for sublist in l for item in sublist]

NAME_BY_ACRONYM = {
    'AL': 'Alabama',        'AK': 'Alaska',        'AZ': 'Arizona',    'AR': 'Arkansas',             'CA': 'California',
    'CO': 'Colorado',       'CT': 'Connecticut',   'DE': 'Delaware',   'DC': 'District of Columbia', 'FL': 'Florida',
    'GA': 'Georgia',        'HI': 'Hawaii',        'ID': 'Idaho',      'IL': 'Illinois',             'IN': 'Indiana',
    'IA': 'Iowa',           'KS': 'Kansas',        'KY': 'Kentucky',   'LA': 'Louisiana',            'ME': 'Maine',
    'MD': 'Maryland',       'MA': 'Massachusetts', 'MI': 'Michigan',   'MN': 'Minnesota',            'MS': 'Mississippi',
    'MO': 'Missouri',       'MT': 'Montana',       'NE': 'Nebraska',   'NV': 'Nevada',               'NH': 'New Hampshire',
    'NJ': 'New Jersey',     'NM': 'New Mexico',    'NY': 'New York',   'NC': 'North Carolina',       'ND': 'North Dakota',
    'OH': 'Ohio',           'OK': 'Oklahoma',      'OR': 'Oregon',     'PA': 'Pennsylvania',         'RI': 'Rhode Island',
    'SC': 'South Carolina', 'SD': 'South Dakota',  'TN': 'Tennessee',  'TX': 'Texas',    'UT':       'Utah',
    'VT': 'Vermont',        'VA': 'Virginia',      'WA': 'Washington', 'WV': 'West Virginia',        'WI': 'Wisconsin',
    'WY': 'Wyoming',        'PR': 'Puerto Rico',
}

def load(file_name=None):
    if file_name:
        with open(file_name) as f:
            return json.load(f)
    
    # Load dataset
    try:
        # From the Guardian on the Web...
        last_update = requests.get('https://interactive.guim.co.uk/2020/11/us-general-election-data/prod/last_updated.json').json()
        last_time = last_update['time']
        url = f'https://interactive.guim.co.uk/2020/11/us-general-election-data/prod/data-out/{last_time}/president_details.json'
        print(last_time)
        data = requests.get(url).json()
        fname = 'data-cache--' + re.compile('[^0-9a-zA-Z-]').sub('.', last_time) + '.json'
        with open(fname, 'w') as f: json.dump(data, f, indent='  ', sort_keys=True)
    except Exception as e:
        # ... or cached
        print(e, file=sys.stderr)
        names = sorted(glob.glob('data-cache--*.json'))
        print('Found cache:', names)
        name = names[-1]
        print('Loading:', name)
        with open(name) as f: data = json.load(f)
    return data

def compute(data):
    electors_by_candidate = {}
    electors_by_candidate_may = {}
    electors_by_state = {}

    new_data = {}
    new_data_short = {}

    n = 0

    # Compute electors by state
    for acronym, state_data in data.items():
        if acronym == 'US': continue
        electors = max( c['electWon'] for c in state_data['candidates'])
        # Some return zero, fix that
        electors_by_state[acronym] = {'AK': 6, 'GA': 16}.get(acronym, electors)

    # Analyze each state
    for acronym, state_data in data.items():
        if acronym == 'US': continue
        n += 1
        extra = {}
        extra_short = {}
        electors = electors_by_state[acronym]

        # Analyze each field
        for k, v in state_data.items():
            if k == 'declarationTime' and v:
                # make it shorter
                dt = dateutil.parser.parse(v)
                state_data[k] = datetime.datetime.strftime(dt, '%b %d %H:%M')
            if k == 'candidates':
                candidate = {c['name'].split()[-1]:c for c in v}
                if n and n < 4:
                    pass #print();print(candidate)
                # compute votes
                nb_counted = sum([c['votes'] for c in v])
                max_votes = int(state_data['totalVotes'] / state_data['reporting'] * 100.)
                rem_votes = max_votes - nb_counted
                # general fields
                #extra['electors'] = electors
                extra['% counted']   = extra_short['% counted']   = state_data['reporting']
                extra['grand total'] = extra_short['grand total'] = max_votes
                extra['uncounted']   = extra_short['uncounted']   = rem_votes
                # per-candidate votes
                extra['- SCORE -'] = ''
                for name, cdata in candidate.items():
                    val = cdata['votes'] / nb_counted
                    extra[name]       = extra_short[name]       = cdata['votes']
                    extra[name + '%']       = format_percent(val)
                    extra_short[name + '%'] = val
                # winner
                winner = sorted([(cdata['votes'], name) for name, cdata in candidate.items()])[-1][1]
                extra['winner'] = winner
                # compute "maybe" winner: who could win, if getting all remaining votes
                extra['- MAYBE -'] = ''
                cand2 = {c['votes']:c['name'].split()[-1] for c in v}
                possibilities = [
                    [[votes2 + (rem_votes if name2!=winner else 0), name2] for votes2, name2 in cand2.items()]
                    for votes1, name1 in cand2.items()]
                winner_may_votes, winner_may = sorted(flattened(possibilities))[-1]
                for name, cdata in candidate.items():
                    val = (cdata['votes']+rem_votes) / (nb_counted+rem_votes)
                    extra[name + '%may'] = format_percent(val) if name != winner else f'({format_percent(val)})'
                extra['may'] = winner_may if winner_may != winner else ''
                #extra['may votes'] = winner_may_votes
                #extra['may %'] = format_percent(winner_may_votes / max_votes)

        # accumulate won electors
        electors_by_candidate.setdefault(winner, 0)
        electors_by_candidate[winner] += electors
        electors_by_candidate_may.setdefault(winner_may, 0)
        electors_by_candidate_may[winner_may] += electors

        # format ints
        for k, v in extra.items():
            if v.__class__ is int:
                extra[k] = f'{v:n}'
        # keep new data and format ints
        new_data[acronym] = extra
        new_data_short[acronym] = extra_short

    electors_by_candidate = [{'Candidate': cand, 'Electors': electors} for cand, electors in electors_by_candidate.items()]
    electors_by_candidate_may = [{'Candidate maybe': cand, 'Electors': electors} for cand, electors in electors_by_candidate_may.items()]
    electors_by_state = {acronym:{'Name': NAME_BY_ACRONYM[acronym], 'Electors': nb} for acronym, nb in electors_by_state.items()}
    
    return new_data, new_data_short, electors_by_candidate, electors_by_candidate_may, electors_by_state

def dic_dict_sub(d1, d2):
    def fmt(v):
        if v.__class__ == int:
            return f'{v:n}'
        elif v.__class__ == float:
            return f'{v*100:.2f}'
        else:
            return v
        
    return { k1:{k2:fmt(v2-d2[k1][k2]) for k2, v2 in v1.items()} for k1, v1 in d1.items()}

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 10000)
pd.set_option('display.max_colwidth', None)

# 1. Results as of Sun 8 Nov

In [2]:
data = load('data-cache--2020-11-08T14.45.31.033.00.00.json')
new_data, new_data_short_0, electors_by_candidate, electors_by_candidate_may, electors_by_state = compute(data)
pd.DataFrame(new_data)

,AK,AL,AR,AZ,CA,CO,CT,DC,DE,FL,GA,HI,IA,ID,IL,IN,KS,KY,LA,MA,MD,ME,MI,MN,MO,MS,MT,NC,ND,NE,NH,NJ,NM,NV,NY,OH,OK,OR,PA,RI,SC,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY
% counted,50,99,99,90,66,95,99,90,99,99,99,96,99,99,92,99,99,99,95,99,86,99,99,99,99,99,99,99,99,99,99,77,99,92,78,96,99,98,99,99,99,94,99,98,93,99,92,85,99,99,97
grand total,381'940,2'313'155,1'220'137,3'659'095,21'489'998,3'340'132,1'788'948,319'751,507'418,11'155'090,5'033'023,597'764,1'700'530,875'985,5'950'636,3'054'370,1'346'982,2'155'455,2'260'329,3'564'678,2'859'958,821'532,5'575'096,3'301'777,3'038'125,1'144'988,606'747,5'513'657,362'294,945'036,809'828,4'486'316,926'419,1'408'439,8'503'584,6'001'302,1'574'370,2'335'332,6'815'155,510'304,2'538'256,449'571,3'076'244,11'470'733,1'397'625,4'419'261,397'220,4'608'967,3'322'701,793'051,283'373
uncounted,194'148,23'131,33'386,365'909,7'446'895,200'994,25'096,34'519,7'209,141'744,50'330,29'835,27'167,18'574,518'060,30'543,13'469,31'625,127'615,53'781,418'071,17'831,79'636,64'282,42'494,20'177,6'067,74'398,3'622,9'450,8'098,1'049'835,17'052,128'007,1'911'701,258'084,27'525,62'723,68'151,7'582,34'148,26'974,57'551,262'556,117'298,44'192,38'154,715'333,43'686,10'477,10'709
- SCORE -,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Trump,118'602,1'430'589,755'815,1'612'585,4'741'191,1'335'253,714'981,15'075,199'829,5'658'847,2'455'305,196'602,896'100,554'019,2'316'566,1'725'723,752'903,1'325'742,1'255'481,1'148'777,857'843,359'502,2'644'525,1'484'671,1'711'848,677'218,341'767,2'733'645,234'845,548'462,365'373,1'385'529,400'095,619'944,2'849'085,3'074'418,1'018'870,928'706,3'314'164,197'421,1'384'852,261'035,1'849'211,5'865'248,759'638,1'946'870,112'688,1'514'563,1'610'030,539'610,193'454
Trump%,63.16,62.47,63.69,48.97,33.76,42.54,40.54,5.29,39.95,51.38,49.28,34.62,53.55,64.62,42.64,57.07,56.46,62.42,58.87,32.72,35.13,44.73,48.12,45.86,57.14,60.21,56.90,50.26,65.48,58.62,45.57,40.32,44.00,48.42,43.22,53.53,65.87,40.87,49.12,39.27,55.30,61.77,61.26,52.33,59.33,44.50,31.38,38.90,49.10,68.95,70.95
Biden,63'992,834'533,417'897,1'631'195,9'163'725,1'753'416,1'029'132,268'625,295'403,5'284'453,2'465'500,365'802,757'707,286'991,3'056'219,1'239'401,551'144,771'884,855'597,2'316'338,1'557'743,430'023,2'790'648,1'717'830,1'242'851,440'284,243'719,2'658'274,114'480,367'319,423'291,2'029'493,496'826,647'474,3'694'996,2'603'731,503'289,1'304'536,3'355'387,300'325,1'091'348,150'467,1'139'666,5'217'656,489'469,2'364'415,242'805,2'303'430,1'630'570,232'502,73'445
Biden%,34.08,36.44,35.21,49.53,65.25,55.86,58.35,94.18,59.06,47.98,49.48,64.41,45.28,33.47,56.26,40.99,41.33,36.34,40.12,65.98,63.79,53.51,50.78,53.06,41.49,39.14,40.57,48.87,31.92,39.26,52.80,59.06,54.63,50.57,56.05,45.34,32.54,57.40,49.73,59.74,43.58,35.61,37.75,46.55,38.23,54.04,67.62,59.16,49.73,29.71,26.94
Jorgensen,5'198,24'902,13'039,49'406,138'187,50'469,19'739,1'532,4'977,70'046,61'888,5'525,19'556,16'401,59'791,58'703,29'466,26'204,21'636,45'782,26'301,14'176,60'287,34'994,40'932,7'309,15'194,47'340,9'347,19'805,13'066,21'459,12'446,13'014,47'802,65'069,24'686,39'367,77'453,4'976,27'908,11'095,29'816,125'273,31'220,63'784,3'573,75'641,38'415,10'462,5'765
Jorgensen%,2.77,1.09,1.10,1.50,0.98,1.61,1.12,0.54,0.99,0.64,1.24,0.97,1.17,1.91,1.10,1.94,2.21,1.23,1.01,1.30,1.08,1.76,1.10,1.08,1.37,0.65,2.53,0.87,2.61,2.12,1.63,0.62,1.37,1.02,0.73,1.13,1.60,1.73,1.15,0.99,1.11,2.63,0.99,1.12,2.44,1.46,1.00,1.94,1.17,1.34,2.11


## Electors

In [3]:
pd.DataFrame(electors_by_candidate)

,Candidate,Electors
0,Trump,219
1,Biden,305


## Potential twist 

In [4]:
pd.DataFrame(electors_by_candidate_may)

,Candidate maybe,Electors
0,Biden,150
1,Trump,374


# 2. Results as of Wed 11 Nov

In [5]:
#data = load('data-cache--2020-11-09T23.58.18.360.00.00.json')
data = load('data-cache--2020-11-11T07.57.21.383.00.00.json')
new_data, new_data_short_1, electors_by_candidate, electors_by_candidate_may, electors_by_state = compute(data)
pd.DataFrame(new_data)

,AK,AL,AR,AZ,CA,CO,CT,DC,DE,FL,GA,HI,IA,ID,IL,IN,KS,KY,LA,MA,MD,ME,MI,MN,MO,MS,MT,NC,ND,NE,NH,NJ,NM,NV,NY,OH,OK,OR,PA,RI,SC,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY
% counted,69,99,99,99,92,95,99,99,99,99,99,96,99,99,92,99,99,99,95,99,93,99,99,99,99,99,99,99,99,99,99,87,99,95,80,96,99,98,99,99,99,94,99,99,95,99,99,99,99,99,97
grand total,348'165,2'313'155,1'220'137,3'382'027,17'359'979,3'340'132,1'839'219,309'639,507'418,11'155'090,5'039'440,597'764,1'702'241,875'985,5'950'539,3'054'370,1'346'982,2'156'561,2'260'329,3'564'678,2'864'067,821'532,5'575'096,3'303'076,3'038'125,1'144'988,606'747,5'522'828,363'525,947'480,809'828,4'525'616,926'419,1'406'744,8'322'739,6'001'302,1'574'370,2'335'332,6'851'606,510'304,2'538'256,449'571,3'076'244,11'356'541,1'477'338,4'441'047,369'134,4'040'926,3'322'701,793'051,283'373
uncounted,111'379,23'131,33'386,33'820,1'557'569,200'994,25'919,5'910,7'209,141'744,50'394,29'835,27'162,18'574,517'963,30'543,13'469,31'641,127'615,53'781,219'569,17'831,79'636,64'146,42'494,20'177,6'067,74'597,3'635,9'474,8'098,609'969,17'052,86'696,1'705'657,258'084,27'525,62'723,68'516,7'582,34'148,26'974,57'551,146'715,95'672,44'410,10'068,65'298,43'686,10'477,10'709
- SCORE -,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Trump,139'264,1'430'589,755'815,1'642'379,5'368'263,1'335'253,713'778,16'306,199'829,5'658'847,2'457'191,196'602,897'140,554'019,2'316'566,1'725'723,752'903,1'326'418,1'255'481,1'148'777,900'080,359'502,2'644'525,1'485'254,1'711'848,677'218,341'767,2'736'929,235'595,550'202,365'373,1'621'452,400'095,634'701,2'859'073,3'074'418,1'018'870,928'706,3'328'801,197'421,1'384'852,261'035,1'849'211,5'865'913,818'784,1'952'834,112'688,1'555'447,1'610'030,539'610,193'454
Trump%,58.81,62.47,63.69,49.05,33.97,42.54,39.36,5.37,39.95,51.38,49.25,34.62,53.56,64.62,42.64,57.07,56.46,62.42,58.87,32.72,34.04,44.73,48.12,45.86,57.14,60.21,56.90,50.24,65.46,58.66,45.57,41.41,44.00,48.08,43.21,53.53,65.87,40.87,49.07,39.27,55.30,61.77,61.26,52.33,59.26,44.42,31.38,39.12,49.10,68.95,70.95
Biden,91'497,834'533,417'897,1'655'192,10'268'890,1'753'416,1'079'223,285'728,295'403,5'284'453,2'469'842,365'802,758'352,286'991,3'056'219,1'239'401,551'144,772'285,855'597,2'316'338,1'715'518,430'023,2'790'648,1'718'683,1'242'851,440'284,243'719,2'663'635,114'902,367'919,423'291,2'267'853,496'826,671'427,3'709'963,2'603'731,503'289,1'304'536,3'376'367,300'325,1'091'348,150'467,1'139'666,5'218'631,527'607,2'379'830,242'805,2'341'901,1'630'570,232'502,73'445
Biden%,38.64,36.44,35.21,49.44,64.98,55.86,59.52,94.07,59.06,47.98,49.51,64.41,45.27,33.47,56.26,40.99,41.33,36.34,40.12,65.98,64.87,53.51,50.78,53.06,41.49,39.14,40.57,48.89,31.93,39.22,52.80,57.92,54.63,50.86,56.07,45.34,32.54,57.40,49.78,59.74,43.58,35.61,37.75,46.55,38.19,54.13,67.62,58.91,49.73,29.71,26.94
Jorgensen,6'025,24'902,13'039,50'636,165'257,50'469,20'299,1'695,4'977,70'046,62'013,5'525,19'587,16'401,59'791,58'703,29'466,26'217,21'636,45'782,28'900,14'176,60'287,34'993,40'932,7'309,15'194,47'667,9'393,19'885,13'066,26'342,12'446,13'920,48'046,65'069,24'686,39'367,77'922,4'976,27'908,11'095,29'816,125'282,35'275,63'973,3'573,78'280,38'415,10'462,5'765
Jorgensen%,2.54,1.09,1.10,1.51,1.05,1.61,1.12,0.56,0.99,0.64,1.24,0.97,1.17,1.91,1.10,1.94,2.21,1.23,1.01,1.30,1.09,1.76,1.10,1.08,1.37,0.65,2.53,0.87,2.61,2.12,1.63,0.67,1.37,1.05,0.73,1.13,1.60,1.73,1.15,0.99,1.11,2.63,0.99,1.12,2.55,1.46,1.00,1.97,1.17,1.34,2.11


## Electors

In [6]:
pd.DataFrame(electors_by_candidate)

,Candidate,Electors
0,Trump,219
1,Biden,305


## Potential twist 

In [7]:
pd.DataFrame(electors_by_candidate_may)

,Candidate maybe,Electors
0,Biden,219
1,Trump,305


# 3. Results as of now

In [8]:
data = load()
new_data, new_data_short_2, electors_by_candidate, electors_by_candidate_may, electors_by_state = compute(data)
pd.DataFrame(new_data)

2020-11-11T20:48:03.210+00:00


,AK,AL,AR,AZ,CA,CO,CT,DC,DE,FL,GA,HI,IA,ID,IL,IN,KS,KY,LA,MA,MD,ME,MI,MN,MO,MS,MT,NC,ND,NE,NH,NJ,NM,NV,NY,OH,OK,OR,PA,RI,SC,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY
% counted,75,99,99,99,92,95,99,99,99,99,99,96,99,99,92,99,99,99,95,99,93,99,99,99,99,99,99,99,99,99,99,89,99,95,80,96,99,98,99,99,99,94,99,99,95,99,99,99,99,99,97
grand total,347'977,2'313'155,1'220'137,3'382'408,17'359'979,3'340'132,1'839'219,309'639,507'418,11'155'090,5'042'276,597'764,1'702'241,875'985,5'950'539,3'054'370,1'346'982,2'156'561,2'260'329,3'564'678,2'864'067,821'532,5'575'096,3'303'076,3'038'125,1'144'988,606'747,5'522'892,363'525,947'480,809'828,4'487'292,926'419,1'407'596,8'338'607,6'001'302,1'574'370,2'335'332,6'859'331,510'304,2'538'256,449'571,3'076'244,11'356'541,1'477'338,4'441'047,369'134,4'040'926,3'322'701,793'051,283'373
uncounted,90'703,23'131,33'386,33'824,1'557'569,200'994,25'919,5'910,7'209,141'744,50'422,29'835,27'162,18'574,517'963,30'543,13'469,31'641,127'615,53'781,219'569,17'831,79'636,64'146,42'494,20'177,6'067,74'597,3'635,9'474,8'098,515'746,17'052,86'752,1'708'923,258'084,27'525,62'723,68'593,7'582,34'148,26'974,57'551,146'715,95'672,44'410,10'068,65'298,43'686,10'477,10'709
- SCORE -,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Trump,148'624,1'430'589,755'815,1'642'664,5'368'263,1'335'253,713'778,16'306,199'829,5'658'847,2'457'856,196'602,897'140,554'019,2'316'566,1'725'723,752'903,1'326'418,1'255'481,1'148'777,900'080,359'502,2'644'525,1'485'254,1'711'848,677'218,341'767,2'736'947,235'595,550'202,365'373,1'649'337,400'095,635'017,2'865'203,3'074'418,1'018'870,928'706,3'331'125,197'421,1'384'852,261'035,1'849'211,5'865'913,818'784,1'952'834,112'688,1'555'447,1'610'030,539'610,193'454
Trump%,57.77,62.47,63.69,49.06,33.97,42.54,39.36,5.37,39.95,51.38,49.24,34.62,53.56,64.62,42.64,57.07,56.46,62.42,58.87,32.72,34.04,44.73,48.12,45.86,57.14,60.21,56.90,50.23,65.46,58.66,45.57,41.53,44.00,48.08,43.22,53.53,65.87,40.87,49.05,39.27,55.30,61.77,61.26,52.33,59.26,44.42,31.38,39.12,49.10,68.95,70.95
Biden,102'080,834'533,417'897,1'655'278,10'268'890,1'753'416,1'079'223,285'728,295'403,5'284'453,2'471'945,365'802,758'352,286'991,3'056'219,1'239'401,551'144,772'285,855'597,2'316'338,1'715'518,430'023,2'790'648,1'718'683,1'242'851,440'284,243'719,2'663'679,114'902,367'919,423'291,2'295'205,496'826,671'887,3'716'314,2'603'731,503'289,1'304'536,3'381'608,300'325,1'091'348,150'467,1'139'666,5'218'631,527'607,2'379'830,242'805,2'341'901,1'630'570,232'502,73'445
Biden%,39.68,36.44,35.21,49.43,64.98,55.86,59.52,94.07,59.06,47.98,49.52,64.41,45.27,33.47,56.26,40.99,41.33,36.34,40.12,65.98,64.87,53.51,50.78,53.06,41.49,39.14,40.57,48.89,31.93,39.22,52.80,57.79,54.63,50.87,56.06,45.34,32.54,57.40,49.80,59.74,43.58,35.61,37.75,46.55,38.19,54.13,67.62,58.91,49.73,29.71,26.94
Jorgensen,6'570,24'902,13'039,50'642,165'257,50'469,20'299,1'695,4'977,70'046,62'053,5'525,19'587,16'401,59'791,58'703,29'466,26'217,21'636,45'782,28'900,14'176,60'287,34'993,40'932,7'309,15'194,47'669,9'393,19'885,13'066,27'004,12'446,13'940,48'167,65'069,24'686,39'367,78'005,4'976,27'908,11'095,29'816,125'282,35'275,63'973,3'573,78'280,38'415,10'462,5'765
Jorgensen%,2.55,1.09,1.10,1.51,1.05,1.61,1.12,0.56,0.99,0.64,1.24,0.97,1.17,1.91,1.10,1.94,2.21,1.23,1.01,1.30,1.09,1.76,1.10,1.08,1.37,0.65,2.53,0.87,2.61,2.12,1.63,0.68,1.37,1.06,0.73,1.13,1.60,1.73,1.15,0.99,1.11,2.63,0.99,1.12,2.55,1.46,1.00,1.97,1.17,1.34,2.11


## Progression 

In [9]:
pd.DataFrame(dic_dict_sub(new_data_short_2, new_data_short_1))

,AK,AL,AR,AZ,CA,CO,CT,DC,DE,FL,GA,HI,IA,ID,IL,IN,KS,KY,LA,MA,MD,ME,MI,MN,MO,MS,MT,NC,ND,NE,NH,NJ,NM,NV,NY,OH,OK,OR,PA,RI,SC,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY
% counted,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
grand total,-188,0,0,381,0,0,0,0,0,0,2'836,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,64,0,0,0,-38'324,0,852,15'868,0,0,0,7'725,0,0,0,0,0,0,0,0,0,0,0,0
uncounted,-20'676,0,0,4,0,0,0,0,0,0,28,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-94'223,0,56,3'266,0,0,0,77,0,0,0,0,0,0,0,0,0,0,0,0
Trump,9'360,0,0,285,0,0,0,0,0,0,665,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,18,0,0,0,27'885,0,316,6'130,0,0,0,2'324,0,0,0,0,0,0,0,0,0,0,0,0
Trump%,-1.05,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.00,0.00,0.00,0.00,0.12,0.00,-0.01,0.01,0.00,0.00,0.00,-0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Biden,10'583,0,0,86,0,0,0,0,0,0,2'103,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,44,0,0,0,27'352,0,460,6'351,0,0,0,5'241,0,0,0,0,0,0,0,0,0,0,0,0
Biden%,1.04,0.00,0.00,-0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.13,0.00,0.00,-0.01,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Jorgensen,545,0,0,6,0,0,0,0,0,0,40,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,662,0,20,121,0,0,0,83,0,0,0,0,0,0,0,0,0,0,0,0
Jorgensen%,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,-0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


## Electors

In [10]:
pd.DataFrame(electors_by_candidate)

,Candidate,Electors
0,Trump,219
1,Biden,305


## Potential twist

In [11]:
pd.DataFrame(electors_by_candidate_may)

,Candidate maybe,Electors
0,Biden,219
1,Trump,305


# 4. Electors per state

In [12]:
pd.DataFrame(electors_by_state).transpose()

,Name,Electors
AK,Alaska,6
AL,Alabama,9
AR,Arkansas,6
AZ,Arizona,11
CA,California,55
CO,Colorado,9
CT,Connecticut,7
DC,District of Columbia,3
DE,Delaware,3
FL,Florida,29
